# Building a model anywhere in the Netherlands  
This example notebook shows a basic example of a model created using online data with the `nlmod` package. `nlmod` contains functions to create modflow models anywhere in the Netherlands.

In [ ]:
import nlmod

In [ ]:
nlmod.util.get_color_logger("INFO")
nlmod.show_versions()

## Model settings
We create a modflow model with the name 'IJmuiden'. This model has the following properties:

- an extent that covers part of the Northsea, Noordzeekanaal and the small port city IJmuiden.
- a structured grid with cells of 100 x 100 m2
- the model is a steady state model with a single time step.
- starting heads of 1 m NAP in every cell.

In [ ]:
# model settings
model_ws = "ijmuiden"
model_name = "IJmuiden"
figdir, cachedir = nlmod.util.get_model_dirs(model_ws)
extent = [95000.0, 105000.0, 494000.0, 500000.0]
delr = 100.0
delc = 100.0
steady_state = True
start_time = "2015-1-1"
starting_head = 1.0

## Download data
First, download the data needed for the model from online sources. It is recommended to store a copy somewhere in case the online dataset changes. For this model we download data from these sources:

- regis (layer model)
- geotop (layer model)
- rijkswaterstaat (surface water)
- jarkus (bathymetry)
- ahn (digital elevation model)
- knmi (precipitation and evaporation)

In [ ]:
# layer models
regis_ds = nlmod.read.regis.download_regis(extent, botm_layer="MSz1",
                                            cachedir=cachedir,
                                            cachename="regis.nc")

geotop_ds = nlmod.read.geotop.download_geotop(extent,
                                              cachedir=cachedir,
                                              cachename="geotop.nc")



In [ ]:
# read surface water data
gdf_surface_water = nlmod.read.rws.get_gdf_surface_water(
    extent=extent, cachedir=cachedir, cachename="rws_surface_water.pklz"
)

# bathymetry
bathymetry_da = nlmod.read.jarkus.download_bathymetry(extent=extent,
                                                      cachedir=cachedir,
                                                      cachename="bathymetry.nc")



In [ ]:
# Digital elevation model
ahn_da = nlmod.read.ahn.download_ahn(extent, cachedir=cachedir, cachename="ahn")


In [ ]:
# knmi data
oc_knmi = nlmod.read.knmi.download_knmi(extent=extent, delr=delr, delc=delc, start='2000-1-1', end='2020-1-1')

## Create model
Now create the modflow model:

- Build a layer model using Regis and Geotop. Use MSz1 as the bottom layer; all deeper Regis layers are excluded. Use `nlmod.read.regis.get_layer_names()` to list available layers.
- Replace the Holocene layer in Regis with Geotop due to missing hydraulic properties.
- Extend Regis/Geotop layers into the North Sea using shoreline extrapolation and Jarkus bathymetry data.
- Add large water bodies (e.g. North Sea, IJsselmeer) as general head boundaries.
- Add surface drainage using the AHN with a conductance of 1000 m²/d.
- Calculate recharge from precipitation and evaporation time series
- Add constant head boundaries on all model edges using the starting head.

In [ ]:
# create layer model
layer_model = nlmod.read.regis.add_geotop_to_regis_layers(regis_ds,
                                                          geotop_ds,
                                                          layers="HLc")

# create a dataset from they layer model
ds = nlmod.to_model_ds(layer_model, model_name, model_ws, delr=delr, delc=delc)

# add time discretisation
ds = nlmod.time.set_ds_time(ds, start=start_time, steady=steady_state, perlen=365 * 5)

# add northsea and bathymetry to modelgrid
ds.update(nlmod.read.rws.discretize_northsea(ds, gdf=gdf_surface_water))
ds.update(nlmod.read.jarkus.discretize_bathymetry(ds, bathymetry_da.drop_vars("time"), cachedir=cachedir, cachename="bathymetry.nc"))
ds = nlmod.dims.add_bathymetry_to_layer_model(ds)

In [ ]:
# create simulation
sim = nlmod.sim.sim(ds)

# create time discretisation
tdis = nlmod.sim.tdis(ds, sim)

# create ims
ims = nlmod.sim.ims(sim)

# create groundwater flow model
gwf = nlmod.gwf.gwf(ds, sim)

# Create discretization
dis = nlmod.gwf.dis(ds, gwf)

# create node property flow
npf = nlmod.gwf.npf(ds, gwf)

# Create the initial conditions package
ic = nlmod.gwf.ic(ds, gwf, starting_head=starting_head)

# Create the output control package
oc = nlmod.gwf.oc(ds, gwf)

In [ ]:
# discretize surface water bodies from geodataframe
rws_ds = nlmod.read.rws.discretize_surface_water(
    ds, gdf=gdf_surface_water, da_basename="rws_oppwater"
)

# add data to model dataset
ds.update(rws_ds)

# build ghb package
ghb = nlmod.gwf.ghb(ds, gwf, bhead="rws_oppwater_stage", cond="rws_oppwater_cond")

In [ ]:
# discretize ahn
ahn_ds = nlmod.read.ahn.discretize_ahn(ds, ahn_da)

# add data to model dataset
ds.update(ahn_ds)

# build surface level drain package
drn = nlmod.gwf.surface_drain_from_ds(ds, gwf, resistance=10.0)

In [ ]:
# add constant head cells at model boundaries
ds.update(nlmod.grid.mask_model_edge(ds))
chd = nlmod.gwf.chd(ds, gwf, mask="edge_mask", head="starting_head")

In [ ]:
# discretize knmi recharge data
knmi_ds = nlmod.read.knmi.discretize_knmi(ds, oc_knmi, cachedir=cachedir, cachename="recharge")

# update model dataset
ds.update(knmi_ds)

# create recharge package
rch = nlmod.gwf.rch(ds, gwf)

Model data is stored in the variable `ds` which is an `xarray.Dataset`.

In [ ]:
ds

## Write and Run
Write the model files and run the model using the function `nlmod.sim.write_and_run)` as shown below. This function has two additional options:

1. Saving the model dataset (`write_ds=True`) for faster future loading.
2. Saving a copy of the current notebook (nb_path="<notebook_name>.ipynb") alongside the model files.

In [ ]:
nlmod.sim.write_and_run(sim, ds, write_ds=True, script_path="01_basic_model.ipynb")

## Visualise
Plot the modelgrid and surface water

In [ ]:
ax = nlmod.plot.modelgrid(ds)
nlmod.plot.surface_water(ds, ax=ax)

Plot model input data.

In [ ]:
fig, axes = nlmod.plot.get_map(ds.extent, nrows=2, ncols=2, figsize=14)
ds["ahn"].plot(ax=axes[0][0])
ds["botm"][0].plot(ax=axes[0][1])
nlmod.layers.get_idomain(ds)[0].plot(ax=axes[1][0])
ds["edge_mask"][0].plot(ax=axes[1][1])

fig, axes = nlmod.plot.get_map(ds.extent, nrows=2, ncols=2, figsize=14)
ds["bathymetry"].plot(ax=axes[0][0])
ds["northsea"].plot(ax=axes[0][1])
ds["kh"][1].plot(ax=axes[1][0])
ds["recharge"].plot(ax=axes[1][1]);